In [1]:
import pandas as pd
import re

In [14]:
df = pd.read_csv('laptop_price.csv', encoding = "ISO-8859-1")
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


Only considering Apple

In [30]:
apple = df[df['Company'] == 'Apple']
apple = df[['Inches','Cpu', 'Ram', 'Memory', 'Price_euros']]
apple.head()

,Inches,Cpu,Ram,Memory,Price_euros
0,13.3,Intel Core i5 2.3GHz,8GB,128GB SSD,1339.69
1,13.3,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,898.94
2,15.6,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,575.00
3,15.4,Intel Core i7 2.7GHz,16GB,512GB SSD,2537.45
4,13.3,Intel Core i5 3.1GHz,8GB,256GB SSD,1803.60


In [31]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Inches       1303 non-null   float64
 1   Cpu          1303 non-null   object 
 2   Ram          1303 non-null   object 
 3   Memory       1303 non-null   object 
 4   Price_euros  1303 non-null   float64
dtypes: float64(2), object(3)
memory usage: 51.0+ KB


In [32]:
#convert data type
def get_cpu(data):
    x = data.str.split()
    L = len(x)
    cpu = []
    for i in range(L):
        row_len = len(x[i])-1
        ghz = re.findall(r"\d+\.?\d*",x[i][row_len])[0]
        cpu.append(ghz) 
    return cpu

def get_ram(data):
    ram = []
    L = len(data)
    for i in range(L):
        gb = re.findall(r"\d+", data[i])[0]
        ram.append(gb)
    return ram

In [33]:
apple['Cpu'] = get_cpu(apple['Cpu'])
apple['Ram'] = get_ram(apple['Ram'])
apple['Memory'] = get_ram(apple['Memory'])

apple['Cpu'] = apple['Cpu'].astype('f')
apple['Ram'] = apple['Ram'].astype('f')
apple['Memory'] = apple['Memory'].astype('f')
apple.head()

/var/folders/wn/y3g3v6ns1xd13wp9v76279qc0000gn/T/ipykernel_31820/2989240344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple['Cpu'] = get_cpu(apple['Cpu'])
/var/folders/wn/y3g3v6ns1xd13wp9v76279qc0000gn/T/ipykernel_31820/2989240344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple['Ram'] = get_ram(apple['Ram'])
/var/folders/wn/y3g3v6ns1xd13wp9v76279qc0000gn/T/ipykernel_31820/2989240344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,Inches,Cpu,Ram,Memory,Price_euros
0,13.3,2.3,8.0,128.0,1339.69
1,13.3,1.8,8.0,128.0,898.94
2,15.6,2.5,8.0,256.0,575.00
3,15.4,2.7,16.0,512.0,2537.45
4,13.3,3.1,8.0,256.0,1803.60


In [34]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Inches       1303 non-null   float64
 1   Cpu          1303 non-null   float32
 2   Ram          1303 non-null   float32
 3   Memory       1303 non-null   float32
 4   Price_euros  1303 non-null   float64
dtypes: float32(3), float64(2)
memory usage: 35.8 KB


In [35]:
from sklearn.linear_model import LinearRegression

In [42]:
y = apple['Price_euros']
X = apple[['Inches','Cpu', 'Ram', 'Memory']]

X.shape, y.shape

((1303, 4), (1303,))

In [45]:
reg = LinearRegression()

#Fitting model with trainig data
reg.fit(X, y)

LinearRegression()

In [46]:
import pickle

# Saving model to disk
pickle.dump(reg, open('model.pkl','wb'))


In [47]:
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
print(model.predict([[13.3, 2.5, 16,128]]))

[1953.52395802]
